In [35]:
#
# hw4pr2digits_modeler:  handwritten-digit modeling with k-nearest neighbors...
#

In [1]:
# libraries!
import numpy as np      # numpy is Python's "array" library
import pandas as pd     # Pandas is Python's "data" library ("dataframe" == spreadsheet)

In [2]:
# let's read in our flower data...
# 
# for read_csv, use header=0 when row 0 is a header row
# 
cleaned_filename = "digits_cleaned.csv"
df_tidy = pd.read_csv(cleaned_filename)   # encoding="utf-8" et al.
print(f"{cleaned_filename} : file read into a pandas dataframe.")
pd.options.display.min_rows = 30
df_tidy

digits_cleaned.csv : file read into a pandas dataframe.


,pix0,pix1,pix2,pix3,pix4,pix5,pix6,pix7,pix8,pix9,...,pix55,pix56,pix57,pix58,pix59,pix60,pix61,pix62,pix63,actual_digit
0,0,0,9,14,8,1,0,0,0,0,...,0,0,0,11,16,15,11,1,0,8
1,0,0,11,12,0,0,0,0,0,2,...,0,0,0,9,12,13,3,0,0,9
2,0,0,1,9,15,11,0,0,0,0,...,0,0,0,1,10,13,3,0,0,0
3,0,0,0,0,14,13,1,0,0,0,...,0,0,0,0,1,13,16,1,0,1
4,0,0,5,12,1,0,0,0,0,0,...,2,0,0,3,11,8,13,12,4,2
5,0,2,9,15,14,9,3,0,0,4,...,0,0,2,12,12,13,11,0,0,3
6,0,0,0,8,15,1,0,0,0,0,...,0,0,0,0,10,15,4,0,0,4
7,0,5,12,13,16,16,2,0,0,11,...,0,0,4,15,16,2,0,0,0,5
8,0,0,0,8,15,1,0,0,0,0,...,2,0,0,0,7,15,16,11,0,6
9,0,0,1,8,15,10,0,0,0,3,...,0,0,0,0,11,9,0,0,0,7


In [3]:
# First, look at the info:
df_tidy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1768 entries, 0 to 1767
Data columns (total 65 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   pix0          1768 non-null   int64
 1   pix1          1768 non-null   int64
 2   pix2          1768 non-null   int64
 3   pix3          1768 non-null   int64
 4   pix4          1768 non-null   int64
 5   pix5          1768 non-null   int64
 6   pix6          1768 non-null   int64
 7   pix7          1768 non-null   int64
 8   pix8          1768 non-null   int64
 9   pix9          1768 non-null   int64
 10  pix10         1768 non-null   int64
 11  pix11         1768 non-null   int64
 12  pix12         1768 non-null   int64
 13  pix13         1768 non-null   int64
 14  pix14         1768 non-null   int64
 15  pix15         1768 non-null   int64
 16  pix16         1768 non-null   int64
 17  pix17         1768 non-null   int64
 18  pix18         1768 non-null   int64
 19  pix19         1768 non-null

In [4]:
A = df_tidy.to_numpy()    # yields the underlying numpy array
A = A.astype('float64')   # so many types:  www.tutorialspoint.com/numpy/numpy_data_types.htm
print(A)

[[ 0.  0.  9. ...  1.  0.  8.]
 [ 0.  0. 11. ...  0.  0.  9.]
 [ 0.  0.  1. ...  0.  0.  0.]
 ...
 [ 0.  0.  1. ...  0.  0.  8.]
 [ 0.  0.  2. ...  0.  0.  9.]
 [ 0.  0. 10. ...  1.  0.  8.]]


In [5]:
#
# Also, nice to have NUM_ROWS and NUM_COLS around
#
NUM_ROWS, NUM_COLS = A.shape
print(f"\nThe dataset has {NUM_ROWS} rows and {NUM_COLS} cols")


The dataset has 1768 rows and 65 cols


In [6]:
# choose a row index, n:
n = 42
COLUMNS = df_tidy.columns
PIXELS = []
COL_INDEX = {}
for i, name in enumerate(COLUMNS):
    COL_INDEX[name] = i  # using the name (as key), look up the value (i)
    PIXELS.append(COLUMNS[i])
print(f"COL_INDEX is {COL_INDEX}\n\n")
print(f"Digit #{n} is {A[n]}\n\n")
print(f"PIXELS is {PIXELS}\n\n")

for i in range(len(COLUMNS)):
    colname = COLUMNS[i]
    value = A[n][i]
    print(f"  Its {colname} is {value}")

COL_INDEX is {'pix0': 0, 'pix1': 1, 'pix2': 2, 'pix3': 3, 'pix4': 4, 'pix5': 5, 'pix6': 6, 'pix7': 7, 'pix8': 8, 'pix9': 9, 'pix10': 10, 'pix11': 11, 'pix12': 12, 'pix13': 13, 'pix14': 14, 'pix15': 15, 'pix16': 16, 'pix17': 17, 'pix18': 18, 'pix19': 19, 'pix20': 20, 'pix21': 21, 'pix22': 22, 'pix23': 23, 'pix24': 24, 'pix25': 25, 'pix26': 26, 'pix27': 27, 'pix28': 28, 'pix29': 29, 'pix30': 30, 'pix31': 31, 'pix32': 32, 'pix33': 33, 'pix34': 34, 'pix35': 35, 'pix36': 36, 'pix37': 37, 'pix38': 38, 'pix39': 39, 'pix40': 40, 'pix41': 41, 'pix42': 42, 'pix43': 43, 'pix44': 44, 'pix45': 45, 'pix46': 46, 'pix47': 47, 'pix48': 48, 'pix49': 49, 'pix50': 50, 'pix51': 51, 'pix52': 52, 'pix53': 53, 'pix54': 54, 'pix55': 55, 'pix56': 56, 'pix57': 57, 'pix58': 58, 'pix59': 59, 'pix60': 60, 'pix61': 61, 'pix62': 62, 'pix63': 63, 'actual_digit': 64}


Digit #42 is [ 0.  0.  0. 10. 11.  0.  0.  0.  0.  0.  9. 16.  6.  0.  0.  0.  0.  0.
 15. 13.  0.  0.  0.  0.  0.  0. 14. 10.  0.  0.  0.  0.  0.  1. 1

In [7]:
print("+++ Start of data definitions +++\n")

#
# we could do this at the data-frame level, too!
#

X_all = A[:,0:64]  # X (features) ... is all rows, columns 0, 1, 2, ..., 63
y_all = A[:,64]    # y (labels) ... is all rows, column 64 only

print(f"y_all (just the labels)   are \n {y_all}")
print(f"X_all (just the features) are \n {X_all}")

+++ Start of data definitions +++

y_all (just the labels)   are 
 [8. 9. 0. ... 8. 9. 8.]
X_all (just the features) are 
 [[ 0.  0.  9. ... 11.  1.  0.]
 [ 0.  0. 11. ...  3.  0.  0.]
 [ 0.  0.  1. ...  3.  0.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]


In [8]:
#
# we scramble the data, to remove (potential) dependence on its ordering: 
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list
# print(indices)

# we scramble both X and y, necessarily with the same permutation
X_labeled = X_all[indices]              # we apply the _same_ permutation to each!
y_labeled = y_all[indices]              # again...
print(f"The scrambled labels are \n {y_labeled}")
print(f"The corresponding data rows are \n {X_labeled}")

The scrambled labels are 
 [1. 5. 0. ... 5. 2. 3.]
The corresponding data rows are 
 [[ 0.  0.  3. ... 16. 15. 11.]
 [ 0.  0.  7. ... 15.  2.  0.]
 [ 0.  0.  6. ...  4.  0.  0.]
 ...
 [ 0.  0.  7. ...  0.  0.  0.]
 [ 0.  0. 10. ...  2.  0.  0.]
 [ 0.  0.  5. ... 10.  0.  0.]]


In [9]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

#
# a common convention:  train on 80%, test on 20%    Let's define the TEST_PERCENT
#

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2) # random_state=42

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

print(f"Held-out data... (testing data: {len(y_test)} rows)")
print(f"y_test: {y_test}")
print(f"X_test (first few rows): {X_test[0:5,:]}")  # 5 rows
print()
print(f"Data used for modeling... (training data: {len(y_train)} rows)")
print(f"y_train: {y_train}")
print(f"X_train (first few rows): {X_train[0:5,:]}")  # 5 rows

training with 1414 rows;  testing with 354 rows

Held-out data... (testing data: 354 rows)
y_test: [5. 6. 8. 7. 3. 0. 6. 0. 5. 2. 0. 9. 2. 3. 9. 4. 2. 2. 7. 6. 7. 7. 0. 3.
 4. 7. 8. 2. 9. 4. 7. 3. 1. 9. 5. 7. 2. 7. 0. 1. 0. 6. 1. 0. 9. 4. 1. 8.
 9. 6. 1. 4. 2. 3. 7. 9. 8. 6. 0. 7. 0. 5. 0. 8. 2. 2. 9. 9. 9. 0. 5. 4.
 8. 9. 9. 8. 7. 5. 1. 3. 1. 3. 4. 4. 0. 1. 2. 8. 4. 9. 8. 0. 9. 4. 2. 8.
 3. 2. 8. 6. 9. 7. 6. 2. 4. 1. 4. 1. 5. 6. 4. 6. 5. 3. 7. 0. 7. 4. 2. 7.
 1. 4. 4. 1. 2. 7. 4. 0. 5. 9. 6. 8. 0. 4. 5. 5. 3. 8. 2. 3. 0. 4. 3. 4.
 5. 2. 3. 9. 3. 4. 3. 0. 6. 5. 2. 7. 3. 1. 7. 2. 7. 4. 5. 6. 2. 4. 7. 0.
 9. 1. 3. 4. 9. 0. 2. 6. 4. 7. 6. 0. 5. 4. 6. 4. 2. 4. 2. 2. 2. 9. 3. 2.
 3. 7. 0. 3. 8. 9. 5. 2. 2. 9. 4. 9. 2. 5. 7. 2. 8. 5. 3. 4. 1. 2. 2. 1.
 6. 6. 6. 4. 7. 6. 1. 4. 7. 8. 3. 6. 2. 5. 6. 5. 9. 9. 8. 1. 4. 0. 3. 3.
 0. 3. 5. 7. 2. 5. 8. 7. 0. 0. 7. 0. 2. 0. 1. 9. 9. 2. 4. 2. 1. 2. 9. 2.
 7. 4. 6. 7. 0. 1. 1. 6. 8. 9. 5. 7. 6. 6. 9. 5. 3. 6. 6. 2. 8. 0. 9. 8.
 0. 5. 8. 5. 9. 1. 4. 6. 

In [10]:
#
# +++ This is the "Model-building and Model-training Cell"
#       
# Create a kNN model and train it! 
#
from sklearn.neighbors import KNeighborsClassifier

k = 1250   # we don't know what k to use, so we guess!  (this will _not_ be a good value)
knn_model = KNeighborsClassifier(n_neighbors=k)       # here, k is the "k" in kNN

# we train the model (it's one line!)
knn_model.fit(X_train, y_train)                              # yay!  trained!
print("Created and trained a knn classifier with k =", k)  

Created and trained a knn classifier with k = 1250


In [11]:
#
# +++ This cell is our "Model-testing Cell"
#
# Now, let's see how well our model does on our "held-out data" (the testing data)
#

# We run our test set:

# the function knn_model.predict is the instantiation of our model
# it's what runs the k-nearest-neighbors algorithm:
predicted_labels = knn_model.predict(X_test)      # THIS IS THE KEY LINE:  predict
actual_labels = y_test

# Let's print them so we can compare...
print("Predicted labels:", predicted_labels)
print("Actual  labels  :", actual_labels)

# And, some overall results
num_correct = sum(predicted_labels == actual_labels)
total = len(actual_labels)
print(f"\nResults on test set:  {num_correct} correct out of {total} total.")

Predicted labels: [5. 6. 8. 8. 1. 3. 6. 3. 5. 5. 3. 3. 3. 3. 3. 6. 5. 3. 3. 5. 5. 3. 6. 3.
 6. 3. 3. 3. 3. 1. 3. 3. 1. 3. 3. 8. 3. 3. 3. 8. 3. 6. 1. 3. 8. 1. 5. 3.
 3. 1. 8. 5. 3. 3. 3. 3. 1. 6. 6. 3. 3. 5. 6. 1. 3. 5. 3. 3. 3. 3. 3. 1.
 8. 3. 3. 3. 5. 5. 1. 3. 1. 3. 1. 6. 3. 8. 3. 1. 8. 3. 3. 5. 3. 5. 3. 5.
 3. 3. 1. 6. 3. 3. 5. 3. 1. 1. 1. 8. 5. 1. 8. 6. 5. 3. 3. 3. 5. 5. 3. 5.
 1. 5. 5. 3. 6. 3. 1. 3. 1. 3. 6. 5. 3. 6. 3. 3. 3. 5. 3. 3. 8. 5. 3. 1.
 5. 3. 3. 3. 3. 1. 3. 8. 6. 5. 3. 5. 3. 3. 8. 3. 8. 1. 5. 6. 3. 6. 5. 3.
 3. 3. 3. 1. 3. 1. 3. 5. 1. 3. 1. 6. 8. 1. 6. 5. 3. 6. 3. 3. 3. 3. 3. 3.
 3. 5. 3. 3. 3. 3. 5. 3. 3. 3. 6. 3. 1. 3. 8. 3. 1. 5. 3. 1. 8. 3. 1. 1.
 6. 6. 1. 1. 3. 5. 1. 1. 8. 3. 3. 6. 3. 3. 1. 3. 3. 3. 3. 8. 1. 3. 3. 3.
 5. 3. 5. 5. 3. 3. 5. 3. 3. 3. 3. 5. 3. 6. 1. 3. 3. 3. 1. 3. 6. 3. 1. 3.
 3. 8. 1. 3. 6. 1. 6. 1. 6. 3. 5. 3. 6. 6. 3. 5. 1. 6. 6. 3. 6. 8. 3. 3.
 3. 3. 3. 5. 3. 1. 1. 5. 3. 8. 3. 5. 1. 3. 6. 6. 3. 5. 3. 1. 5. 8. 6. 3.
 3. 6. 5. 3. 3. 3. 3. 5. 1. 1. 5.

In [12]:
#
# Let's print things in a vertical table
#

def compare_labels(predicted_labels, actual_labels):
    """ a more neatly formatted comparison """
    NUM_LABELS = len(predicted_labels)
    num_correct = 0

    print()
    print(f'row {"#":>3s} : {"predicted":>12s} {"actual":<12s}   {"result"}')   
    
    for i in range(NUM_LABELS):
        p = int(round(predicted_labels[i]))         # round protects from fp error 
        a = int(round(actual_labels[i]))
        result = "incorrect"
        if p == a:  # if they match,
            result = ""       # no longer incorrect
            num_correct += 1  # and we count a match!

        print(f"row {i:>3d} : {p:>12d} {a:<12d}   {result}")   

    print()
    print("Correct:", num_correct, "out of", NUM_LABELS)
    return num_correct

# let's try it out!
compare_labels(predicted_labels,actual_labels)


row   # :    predicted actual         result
row   0 :            5 5              
row   1 :            6 6              
row   2 :            8 8              
row   3 :            8 7              incorrect
row   4 :            1 3              incorrect
row   5 :            3 0              incorrect
row   6 :            6 6              
row   7 :            3 0              incorrect
row   8 :            5 5              
row   9 :            5 2              incorrect
row  10 :            3 0              incorrect
row  11 :            3 9              incorrect
row  12 :            3 2              incorrect
row  13 :            3 3              
row  14 :            3 9              incorrect
row  15 :            6 4              incorrect
row  16 :            5 2              incorrect
row  17 :            3 2              incorrect
row  18 :            3 7              incorrect
row  19 :            5 6              incorrect
row  20 :            5 7              incorrect


84

In [13]:
def predictive_model( Features ):
    """ input: a list of pixel values from pix0 to pix64     
        output: the predicted integer value created from these pixel values
    """
    our_features = np.asarray([Features])                 # extra brackets needed
    predicted_digits = knn_model.predict(our_features)   # PREDICT!
    return float(predicted_digits)

#
# Try it!
# 
# Features = eval(input("Enter new Features: "))
#
Features = [0,0,0,8,14,0,0,0,0,0,5,16,11,0,0,0,0,1,15,14,1,6,0,0,0,7,16,5,3,16,8,0,0,8,16,8,14,16,2,0,0,0,6,14,16,11,0,0,0,0,0,6,16,4,0,0,0,0,0,10,15,0,0,0]       
result = predictive_model( Features )
print(f"I predict {result} from Features {Features}")

I predict 1.0 from Features [0, 0, 0, 8, 14, 0, 0, 0, 0, 0, 5, 16, 11, 0, 0, 0, 0, 1, 15, 14, 1, 6, 0, 0, 0, 7, 16, 5, 3, 16, 8, 0, 0, 8, 16, 8, 14, 16, 2, 0, 0, 0, 6, 14, 16, 11, 0, 0, 0, 0, 0, 6, 16, 4, 0, 0, 0, 0, 0, 10, 15, 0, 0, 0]


In [14]:
#
# Here, we use "cross validation" to find the "best" k...
#

from sklearn.model_selection import cross_val_score

#
# cross-validation splits the training set into two pieces:
#   + model-building and model-validation. We'll use "build" and "validate"
#
best_k = 500  # Not correct!
best_accuracy = 0.0  # also not correct...

# Note that we are cross-validating using only our TEST data!
for k in range(1,501):
    knn_cv_model = KNeighborsClassifier(n_neighbors=k)   # build a knn_model for every k
    cv_scores = cross_val_score( knn_cv_model, X_train, y_train, cv=5 )  # cv=5 means 80/20
    this_cv_accuracy = cv_scores.mean()               # mean() is numpy's built-in average function 
    print(f"k: {k:2d}  cv accuracy: {this_cv_accuracy:7.4f}")

    if this_cv_accuracy > best_accuracy:  # is this one better?
        best_accuracy = this_cv_accuracy  # track the best accuracy
        best_k = k                        # with the best k

    
# use best_k!
print(f"best_k = {best_k}   yields the highest average cv accuracy.")  # print the best one


k:  1  cv accuracy:  0.9866
k:  2  cv accuracy:  0.9859
k:  3  cv accuracy:  0.9873
k:  4  cv accuracy:  0.9837
k:  5  cv accuracy:  0.9873
k:  6  cv accuracy:  0.9830
k:  7  cv accuracy:  0.9837
k:  8  cv accuracy:  0.9823
k:  9  cv accuracy:  0.9809
k: 10  cv accuracy:  0.9795
k: 11  cv accuracy:  0.9802
k: 12  cv accuracy:  0.9774
k: 13  cv accuracy:  0.9767
k: 14  cv accuracy:  0.9738
k: 15  cv accuracy:  0.9724
k: 16  cv accuracy:  0.9696
k: 17  cv accuracy:  0.9710
k: 18  cv accuracy:  0.9675
k: 19  cv accuracy:  0.9646
k: 20  cv accuracy:  0.9625
k: 21  cv accuracy:  0.9632
k: 22  cv accuracy:  0.9597
k: 23  cv accuracy:  0.9618
k: 24  cv accuracy:  0.9611
k: 25  cv accuracy:  0.9597
k: 26  cv accuracy:  0.9583
k: 27  cv accuracy:  0.9597
k: 28  cv accuracy:  0.9604
k: 29  cv accuracy:  0.9562
k: 30  cv accuracy:  0.9547
k: 31  cv accuracy:  0.9540
k: 32  cv accuracy:  0.9540
k: 33  cv accuracy:  0.9519
k: 34  cv accuracy:  0.9512
k: 35  cv accuracy:  0.9519
k: 36  cv accuracy: 

In [15]:
#
# With the best k, we build and train a new model:
#
# Now using best_k instead of the original, randomly-guessed value:   
#
best_k = best_k   # not needed, but nice
from sklearn.neighbors import KNeighborsClassifier
knn_model_tuned = KNeighborsClassifier(n_neighbors=best_k)   # here, we use the best_k!

# we train the model (one line!)
knn_model_tuned.fit(X_train, y_train)                              # yay!  trained!
print(f"Created + trained a knn classifier, now tuned with a (best) k of {best_k}")  

# How does it do?!  The next cell will show...

Created + trained a knn classifier, now tuned with a (best) k of 3


In [16]:
#
# Re-create and re-run the  "Model-testing Cell"     How does it do with best_k?!
#
predicted_labels = knn_model_tuned.predict(X_test)
actual_labels = y_test

# Let's print them so we can compare...
print("Predicted labels:", predicted_labels)
print("Actual labels:", actual_labels)

# And, the overall results
num_correct = sum(predicted_labels == actual_labels)
total = len(actual_labels)
print(f"\nResults on test set:  {num_correct} correct out of {total} total.\n\n")

# Plus, we'll print our nicer table...
compare_labels(predicted_labels,actual_labels)

Predicted labels: [5. 6. 8. 7. 3. 0. 6. 0. 5. 2. 0. 9. 2. 3. 9. 4. 2. 2. 7. 6. 7. 7. 0. 3.
 4. 7. 8. 2. 9. 4. 7. 3. 1. 9. 5. 7. 2. 7. 0. 1. 0. 6. 1. 0. 9. 4. 1. 8.
 9. 6. 1. 4. 2. 3. 7. 9. 8. 6. 0. 7. 0. 5. 0. 8. 2. 2. 9. 9. 9. 0. 5. 4.
 8. 4. 9. 8. 7. 5. 1. 3. 1. 3. 4. 4. 0. 1. 2. 8. 4. 9. 8. 0. 9. 4. 2. 8.
 3. 2. 1. 6. 9. 7. 6. 2. 4. 1. 4. 1. 5. 6. 4. 6. 5. 3. 7. 0. 7. 4. 2. 7.
 1. 4. 4. 1. 2. 7. 4. 0. 5. 9. 6. 8. 0. 4. 5. 5. 3. 8. 2. 3. 0. 4. 3. 4.
 5. 2. 3. 9. 3. 4. 3. 0. 6. 5. 2. 7. 3. 1. 7. 2. 7. 4. 5. 6. 2. 4. 7. 0.
 9. 1. 3. 4. 9. 0. 2. 6. 4. 7. 6. 0. 5. 4. 6. 4. 2. 4. 2. 2. 2. 9. 3. 2.
 3. 7. 0. 3. 8. 9. 5. 2. 2. 9. 4. 9. 2. 5. 7. 2. 8. 5. 3. 4. 1. 2. 2. 1.
 6. 6. 6. 4. 7. 6. 1. 4. 7. 8. 3. 6. 2. 5. 6. 5. 9. 9. 8. 1. 4. 0. 3. 3.
 0. 3. 5. 7. 2. 5. 8. 7. 0. 0. 7. 0. 2. 0. 1. 9. 9. 2. 4. 2. 1. 2. 9. 2.
 7. 4. 6. 7. 0. 1. 1. 6. 8. 9. 5. 7. 6. 6. 9. 5. 3. 6. 6. 2. 8. 0. 9. 8.
 0. 5. 8. 5. 9. 1. 4. 6. 3. 4. 2. 6. 4. 9. 8. 6. 3. 6. 0. 1. 8. 6. 6. 0.
 5. 3. 5. 7. 3. 0. 9. 2. 4. 4. 5.

351

In [17]:
#
# Ok!  We tuned our knn modeling to use the "best" value of k...
#
# And, we should now use ALL available data to train our final predictive model:
#
knn_model_final = KNeighborsClassifier(n_neighbors=best_k)     # here, we use the best_k
knn_model_final.fit(X_all, y_all)                              # KEY DIFFERENCE:  we use ALL the data!
print(f"Created + trained a 'final' knn classifier, with a (best) k of {best_k}") 

Created + trained a 'final' knn classifier, with a (best) k of 3


In [18]:
#
# final predictive model (k-nearest-neighbor), with tuned k + ALL data incorporated
#

def predictive_model( Features, Model ):
    """ input: a list of 64 features (one of each of the 64 pixels)
                [pix0, pix1, ..., pix64 ]
        output: the predicted digit that is formed from the 64 pixels
                  0, 1, 2, 3, ... , or 9
    """
    our_features = np.asarray([Features])                 # extra brackets needed
    predicted_digit = Model.predict(our_features)       # The model's prediction!
    predicted_digit = int(round(predicted_digit[0]))  # unpack the extra brackets
    return predicted_digit
   
#
# Try it!
# 
# Features = eval(input("Enter new Features: "))
#

LoD = [[0,0,0,8,14,0,0,0,0,0,5,16,11,0,0,0,0,1,15,14,1,6,0,0,0,7,16,5,3,16,8,0,0,8,16,8,14,16,2,0,0,0,6,14,16,11,0,0,0,0,0,6,16,4,0,0,0,0,0,10,15,0,0,0],
[0,0,0,5,14,12,2,0,0,0,7,15,8,14,4,0,0,0,6,2,3,13,1,0,0,0,0,1,13,4,0,0,0,0,1,11,9,0,0,0,0,8,16,13,0,0,0,0,0,5,14,16,11,2,0,0,0,0,0,6,12,13,3,0],
[0,0,0,3,16,3,0,0,0,0,0,12,16,2,0,0,0,0,8,16,16,4,0,0,0,7,16,15,16,12,11,0,0,8,16,16,16,13,3,0,0,0,0,7,14,1,0,0,0,0,0,6,16,0,0,0,0,0,0,4,14,0,0,0],
[0,0,0,3,15,10,1,0,0,0,0,11,10,16,4,0,0,0,0,12,1,15,6,0,0,0,0,3,4,15,4,0,0,0,0,6,15,6,0,0,0,4,15,16,9,0,0,0,0,0,13,16,15,9,3,0,0,0,0,4,9,14,7,0],
[0,0,0,3,16,3,0,0,0,0,0,10,16,11,0,0,0,0,4,16,16,8,0,0,0,2,14,12,16,5,0,0,0,10,16,14,16,16,11,0,0,5,12,13,16,8,3,0,0,0,0,2,15,3,0,0,0,0,0,4,12,0,0,0],
[0,0,7,15,15,4,0,0,0,8,16,16,16,4,0,0,0,8,15,8,16,4,0,0,0,0,0,10,15,0,0,0,0,0,1,15,9,0,0,0,0,0,6,16,2,0,0,0,0,0,8,16,8,11,9,0,0,0,9,16,16,12,3,0]]



# run on each one:
for Features in LoD:
    predicted_pixels = predictive_model( Features, knn_model_final )  # pass in the model, too
    print(f"I predict {predicted_pixels} from the features {Features}")    
    print()

I predict 4 from the features [0, 0, 0, 8, 14, 0, 0, 0, 0, 0, 5, 16, 11, 0, 0, 0, 0, 1, 15, 14, 1, 6, 0, 0, 0, 7, 16, 5, 3, 16, 8, 0, 0, 8, 16, 8, 14, 16, 2, 0, 0, 0, 6, 14, 16, 11, 0, 0, 0, 0, 0, 6, 16, 4, 0, 0, 0, 0, 0, 10, 15, 0, 0, 0]

I predict 2 from the features [0, 0, 0, 5, 14, 12, 2, 0, 0, 0, 7, 15, 8, 14, 4, 0, 0, 0, 6, 2, 3, 13, 1, 0, 0, 0, 0, 1, 13, 4, 0, 0, 0, 0, 1, 11, 9, 0, 0, 0, 0, 8, 16, 13, 0, 0, 0, 0, 0, 5, 14, 16, 11, 2, 0, 0, 0, 0, 0, 6, 12, 13, 3, 0]

I predict 4 from the features [0, 0, 0, 3, 16, 3, 0, 0, 0, 0, 0, 12, 16, 2, 0, 0, 0, 0, 8, 16, 16, 4, 0, 0, 0, 7, 16, 15, 16, 12, 11, 0, 0, 8, 16, 16, 16, 13, 3, 0, 0, 0, 0, 7, 14, 1, 0, 0, 0, 0, 0, 6, 16, 0, 0, 0, 0, 0, 0, 4, 14, 0, 0, 0]

I predict 2 from the features [0, 0, 0, 3, 15, 10, 1, 0, 0, 0, 0, 11, 10, 16, 4, 0, 0, 0, 0, 12, 1, 15, 6, 0, 0, 0, 0, 3, 4, 15, 4, 0, 0, 0, 0, 6, 15, 6, 0, 0, 0, 4, 15, 16, 9, 0, 0, 0, 0, 0, 13, 16, 15, 9, 3, 0, 0, 0, 0, 4, 9, 14, 7, 0]

I predict 4 from the features [0, 0, 0, 3,

<br>

#### Digit-recognition experiment

In [36]:
def masked_digits(pixel_num):
    """ Takes in a list of pixel values returns another list with pixel_num number of elements/pixels"""

    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import cross_val_score
    from sklearn.neighbors import KNeighborsClassifier

    X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2) # random_state=42

    X_train = X_train[:,:pixel_num] # We want pixel_num amount of pixels...
    X_test = X_test[:,:pixel_num] # We want pixel_num amount of pixels...
    y_train = y_train # These do not change since we still want all possible output digits
    y_test = y_test # These do not change since we still want all possible output digits
    
    #
    # cross-validation splits the training set into two pieces:
    #   + model-building and model-validation. We'll use "build" and "validate"
    #
    best_k = 42  # Not correct!
    best_accuracy = 0.0  # also not correct...

    # Note that we are cross-validating using only our TEST data!
    for k in range(1,43):
        knn_cv_model = KNeighborsClassifier(n_neighbors=k)   # build a knn_model for every k
        cv_scores = cross_val_score( knn_cv_model, X_train, y_train, cv=5 )  # cv=5 means 80/20
        this_cv_accuracy = cv_scores.mean()               # mean() is numpy's built-in average function 
        # print(f"k: {k:2d}  cv accuracy: {this_cv_accuracy:7.4f}")

        if this_cv_accuracy > best_accuracy:  
            best_accuracy = this_cv_accuracy  # track the best accuracy
            best_k = k                        # with the best k


    best_k = best_k   # not needed, but nice
    knn_model_tuned = KNeighborsClassifier(n_neighbors=best_k)   # here, we use the best_k!

    # we train the model (one line!)
    knn_model_tuned.fit(X_train, y_train)                              # yay!  trained!

    predicted_labels = knn_model_tuned.predict(X_test)
    actual_labels = y_test

    # And, the overall results
    num_correct = sum(predicted_labels == actual_labels)
    total = len(actual_labels)
    testing_accuracy = num_correct / total
    # print(f"\nResults on test set:  {num_correct} correct out of {total} total.\n\n")

    # this is the output we want to insert in our table
    return best_k, testing_accuracy
    
    
    

masked_digits(42)


(3, 0.940677966101695)

In [34]:

# importing required library
# got this code online (https://www.geeksforgeeks.org/generate-simple-ascii-tables-using-prettytable-in-python/)
from prettytable import PrettyTable
  
# creating an empty PrettyTable
x = PrettyTable()
  
# adding data into the table
# row by row
x.field_names = ["How Many Pixels to Use", "Best_K", "Testing Accuracy"]
for i in range(8, 65):
    values = masked_digits(i)
    x.add_row([f'{i}', f'{values[0]}', f'{values[1]}'])

# printing generated table
# Open in text editor to see full results
print(x)

+------------------------+--------+--------------------+
| How Many Pixels to Use | Best_K |  Testing Accuracy  |
+------------------------+--------+--------------------+
|           8            |   30   | 0.3983050847457627 |
|           9            |   29   | 0.4293785310734463 |
|           10           |   18   | 0.4180790960451977 |
|           11           |   25   | 0.5084745762711864 |
|           12           |   15   | 0.5451977401129944 |
|           13           |   13   | 0.5508474576271186 |
|           14           |   14   | 0.6186440677966102 |
|           15           |   16   | 0.6299435028248588 |
|           16           |   11   | 0.6440677966101694 |
|           17           |   5    | 0.6836158192090396 |
|           18           |   14   | 0.652542372881356  |
|           19           |   10   | 0.6666666666666666 |
|           20           |   14   | 0.7372881355932204 |
|           21           |   3    | 0.748587570621469  |
|           22           |   7 